In [ ]:
# https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1 を参照
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3==1.0.3 > /dev/null
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install transformers
!pip install jaconv
!pip install unicodedata
!pip install neologdn

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/MyDrive/Joint_Inference")

In [ ]:
# フォルダを作成
! mkdir resource BERT
# 万病辞書のダウンロード
! wget -O ./resource/MANBYO_201907_Dic-utf8.dic  http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic 
# UTH-BERTのダウンロード
! wget -O ./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
# ファイルを解凍
! unzip ./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip -d ./BERT
# モデルのダウンロード
! wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Ec8P255DFB1KJkaZYO_MZAyLE9CXxrp6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ec8P255DFB1KJkaZYO_MZAyLE9CXxrp6" -O ./models/20211012/model.pt && rm -rf /tmp/cookies.txt

In [ ]:
from lib.models import BERT_JOINT
from lib.util import load_model, decode_ner, decode_re, spacy2df
from lib.preprocess import uth_bert_preprocess, mecab_wakati, make_vector
import os, sys, random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
from transformers import BertModel, BertTokenizer
import spacy
from spacy import displacy
from lib.html import transform_result2html_re, transform_result2html_ner
from IPython.display import HTML
from dataclasses import dataclass

In [ ]:
# https://github.com/WindChimeRan/pytorch_multi_head_selection_re を参照
@dataclass
class Parameters(object):
  bert_path: str = "./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"
  neologd_path: str = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
  manbyo_path: str = "./resource/MANBYO_201907_Dic-utf8.dic"
  rel2idx_path: str = "./models/20211012/rel2idx.json"
  tag2idx_path: str = "./models/20211012/tag2idx.json"
  model_path: str = "./models/20211012/model.pt"
  max_text_len: int = 510

In [ ]:
# 情報抽出を行う文字列
text = "患者:55歳,男性.事務員.\n主訴:嘔気.\n既往歴:特記すべきことなし."

In [ ]:
paras = Parameters()
text = uth_bert_preprocess(text)
tokens = mecab_wakati(text, paras)
vecs = make_vector(tokens, paras)
model, tag2idx, rel2idx = load_model(paras)
ner_preds, re_preds = model(torch.tensor(vecs), rel2idx)
ner_preds = decode_ner(ner_preds, tag2idx)
spacy_format = decode_re(tokens, re_preds, ner_preds, rel2idx)
res_df = spacy2df(spacy_format, "result.csv")
re_htmls = transform_result2html_re(res_df, spacy_format)
ner_htmls = transform_result2html_ner(res_df)

In [ ]:
HTML(ner_htmls)

In [ ]:
HTML(re_htmls)

In [ ]:
res_df